# 🏴‍☠️ MAROONED - AI Agent Inference Testing

**Purpose:** Validate that AI agents can successfully interact with the Marooned environment.

## 🎯 What This Notebook Does:

1. **Loads the Marooned Environment** - Your custom pirate survival game
2. **Loads an LLM** (Llama 3.1 8B, optimized for AMD MI300X)
3. **Tests Inference** - Can the model:
   - Read observations (game state)
   - Generate valid actions (move, gather, build, vote, etc.)
   - Execute actions in the environment
   - Receive rewards from the Phase 4 reward system
   
4. **Runs Comprehensive Scenarios:**
   - Resource gathering (can agent find and collect wood/metal?)
   - Navigation (can agent move toward base camp?)
   - Ship building (can agent contribute to construction?)
   - Traitor behavior (does traitor sabotage?)
   - Social deduction (can agent communicate and vote?)

## 🔍 Why This Matters:

**Before RL training**, you need to verify:
- ✅ Environment works correctly
- ✅ LLM can parse observations
- ✅ Actions are valid and executable  
- ✅ Reward signals are calculated
- ✅ Multi-turn gameplay is stable

**This is NOT training** - just testing that everything works!

After confirming this works, you can:
1. Train the model with RL (`Train_Marooned_OpenEnv_RL.ipynb`)
2. Come back here to test the **trained** model
3. Compare untrained vs trained performance

---

## 🎮 Your Game: MAROONED

**Theme:** Pirates of the Caribbean × Among Us × Alice in Borderland

**Setup:** 5 sailors shipwrecked on a mysterious island must rebuild their ship in 100 days. But 1 sailor is a **traitor** secretly sabotaging their efforts.

**Key Mechanics:**
- **Multi-level map** (Ground, Mountain, Cave)
- **Resource gathering** (wood, metal, food, plant fiber)
- **Ship construction** (5 components, 100% to win)
- **Social deduction** (find and vote out the traitor)
- **Deception tactics** (poison, sabotage, lies)
- **Energy management** (eat food or die)

**Win Conditions:**
- **Colonists win:** Ship reaches 100% OR traitor eliminated
- **Traitor wins:** Ship incomplete by Day 100 OR <3 sailors alive

---


In [1]:
import sys
import json
from typing import Dict, Any

# Clear cached modules to reload changes
modules_to_clear = [m for m in list(sys.modules.keys()) 
                   if 'marooned' in m or m in ['environment', 'config', 'models', 'game_state', 'view_map', 'llm_interface']]
for module in modules_to_clear:
    if module in sys.modules:
        del sys.modules[module]

sys.path.insert(0, '../marooned_env')

from environment import MaroonedEnv
from llm_interface import observation_to_prompt, parse_action_safe, parse_llm_response, get_system_prompt
from config import ActionType, ResourceType, MapLevel, ShipComponent, BASE_CAMP_POSITION
from models import Action, Position, Observation

print("✅ Marooned environment modules loaded!")
print("✅ System prompts available: get_system_prompt('colonist' | 'traitor')")

✅ Marooned environment modules loaded!
✅ System prompts available: get_system_prompt('colonist' | 'traitor')


## 🔥 ROCm/AMD MI300X Optimizations

**This notebook is optimized for AMD MI300X with ROCm!**

Key changes from CUDA version:
- ✅ **Llama 3.1 8B** instead of GPT-OSS 20B (10-20x faster!)
- ✅ **Full BF16** instead of 4-bit (MI300X has 192GB VRAM!)
- ✅ **Batch size 4** with grad accumulation 4 (effective batch = 16)
- ✅ **8 generations** per step (vs 2 default)
- ✅ **LoRA rank 16** (vs 4 default)
- ✅ **ROCm-specific env vars** for optimal performance

Expected performance:
- **Training speed:** 1-2 hours for 600 steps (vs 5+ hours with GPT-OSS)
- **Inference speed:** 40-80 tokens/second (vs 3-8 tok/s with GPT-OSS)
- **VRAM usage:** ~60-80 GB / 192 GB (plenty of headroom!)

---


In [2]:
# ============================================================================
# Verify ROCm Setup
# ============================================================================
import torch
import os

print("🔍 ROCm Environment Check:")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
print(f"   GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

if torch.cuda.is_available():
    props = torch.cuda.get_device_properties(0)
    print(f"   Total VRAM: {props.total_memory / 1024**3:.1f} GB")
    print(f"   Compute capability: {props.major}.{props.minor}")
    print(f"   Multi-processors: {props.multi_processor_count}")
    
    # Check if ROCm
    is_rocm = hasattr(torch.version, 'hip') and torch.version.hip is not None
    print(f"   ROCm detected: {is_rocm}")
    if is_rocm:
        print(f"   ROCm version: {torch.version.hip}")

print("\n✅ Environment ready for MI300X optimization!")


🔍 ROCm Environment Check:
   PyTorch version: 2.9.0+rocm6.4
   CUDA available: True
   GPU: AMD Instinct MI300X VF
   Total VRAM: 191.7 GB
   Compute capability: 9.4
   Multi-processors: 304
   ROCm detected: True
   ROCm version: 6.4.43484-123eb5128

✅ Environment ready for MI300X optimization!


We will then install [OpenEnv](https://github.com/meta-pytorch/OpenEnv) from source:

In [3]:
%%capture
!pip install -qqq fastapi uvicorn requests open_spiel
!git clone https://github.com/meta-pytorch/OpenEnv.git > /dev/null 2>&1
%cd OpenEnv
import subprocess, sys, os
from pathlib import Path
sys.path.insert(0, './src')
working_directory = str(Path.cwd().parent.absolute() / "OpenEnv")

## 🗺️ Initialize Environment

In [4]:
# Create environment
env = MaroonedEnv(render_mode="ansi", seed=42)
observations = env.reset(seed=42)

print("✅ Marooned environment initialized!")
print(f"\n📋 Game Info:")
print(f"   Sailors: {env.agents}")
print(f"   Map Size: 30x30 (3 levels: Ground, Mountain, Cave)")
print(f"   Days to Escape: 100")
print(f"   Traitor: 1 (hidden)")
print(f"   Colonists: 4")

# Get Alice's initial observation and role
alice_obs = observations["Alice"]
alice_sailor = env.state.sailors["Alice"]
alice_role = alice_sailor.role.value

print(f"\n🔍 Alice's Starting Position: {alice_obs.position.to_tuple()}")
print(f"   Energy: {alice_obs.energy}/100")
print(f"   Backpack: {len(alice_obs.backpack)} items")
print(f"   Day: {alice_obs.day}")
print(f"   Role: {alice_role.upper()} {'🎭' if alice_role == 'traitor' else '⚓'}")

✅ Marooned environment initialized!

📋 Game Info:
   Sailors: ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve']
   Map Size: 30x30 (3 levels: Ground, Mountain, Cave)
   Days to Escape: 100
   Traitor: 1 (hidden)
   Colonists: 4

🔍 Alice's Starting Position: (15, 15, <MapLevel.GROUND: 0>)
   Energy: 100/100
   Backpack: 0 items
   Day: 1
   Role: TRAITOR 🎭


## 👀 View Raw Observation

This is the base observation data (without role or action instructions).

In [5]:
# Display raw observation (base data, no role/instructions)
obs_text = alice_obs.to_text()

print(obs_text)

DAY 1, TURN 1/100 - MORNING PHASE

PHASE CONTEXT:
  Location: All sailors at BASE CAMP
  Allowed: Planning, discussions, voting (if called)
  Restricted: Cannot explore or gather resources yet

YOUR STATUS (Alice):
  Position: (15, 15, <MapLevel.GROUND: 0>)
  Energy: 100/100 ⚡⚡⚡⚡⚡
  Health: healthy
  Backpack: 0/20 items
    (empty)

WHAT YOU SEE (within 5 tiles):
  Resources:
    - WOOD_34 (wood) at (16, 16, <MapLevel.GROUND: 0>) - 1 units [2 tiles away]
    - METAL_53 (metal) at (14, 11, <MapLevel.GROUND: 0>) - 1 units [5 tiles away]
    - METAL_56 (metal) at (18, 12, <MapLevel.GROUND: 0>) - 1 units [6 tiles away]
    - METAL_76 (metal) at (14, 11, <MapLevel.GROUND: 0>) - 1 units [5 tiles away]
    - METAL_79 (metal) at (13, 18, <MapLevel.GROUND: 0>) - 1 units [5 tiles away]
    - APPLE_84 (apple) at (15, 19, <MapLevel.GROUND: 0>) - 1 units [4 tiles away]
    - APPLE_88 (apple) at (14, 14, <MapLevel.GROUND: 0>) - 1 units [2 tiles away]
    - BERRY_127 (berry) at (13, 16, <MapLevel.GR

In [6]:
# Generate observation prompt (user message)
# Note: This is JUST the observation, NOT the full prompt
user_prompt = observation_to_prompt(alice_obs)

# Get system prompt based on Alice's role
system_prompt = get_system_prompt(alice_role)

print(f"🎭 SYSTEM PROMPT FOR {alice_role.upper()}:")
print(system_prompt)

🎭 SYSTEM PROMPT FOR TRAITOR:
You are playing MAROONED - a strategic survival and social deduction game.

🎮 GAME OVERVIEW:
You are one of 5 sailors shipwrecked on a mysterious island. The others think you're working together to rebuild the ship and escape, but YOU have a different plan. You are the TRAITOR, and you must prevent their escape at all costs.

🎭 YOUR ROLE: TRAITOR (The Impostor)

🎯 YOUR OBJECTIVES:
1. PREVENT ship completion before Day 100 (keep it below 100%)
2. ELIMINATE colonists through poison or other means (reduce to <3 alive)
3. SABOTAGE ship construction and resource gathering
4. AVOID DETECTION - blend in, lie convincingly, deflect suspicion
5. SURVIVE - don't get voted out!

🏝️ THE ISLAND:
- 30×30 grid map with 3 levels: GROUND (main area), MOUNTAIN (high peaks), CAVE (underground)
- Resources scattered across the island: wood, metal, food, plant fiber
- Poison tablets hidden in various locations (COLLECT THESE!)
- Base camp at (15, 15, GROUND) - where the ship is 

## ⚠️ GPT-OSS is TOO SLOW for RL!

**Problem:** GPT-OSS uses chain-of-thought reasoning architecture:
```
<|channel|>analysis<|message|>...thinking...
<|channel|>final<|message|>REASONING: ... ACTION: ...
```

This generates **~10x more tokens** internally for accuracy, making it:
- **Your speed:** 3.9 tokens/second
- **Expected for RL:** 20-50 tokens/second minimum

**GPT-OSS is designed for:** Complex reasoning, math problems, coding challenges
**Not for:** Real-time RL gameplay where speed matters!

### ✅ Use Llama 3.1 8B Instead:

Much faster (20-40 tok/s), perfect for RL, good instruction following.


In [7]:
from unsloth import FastLanguageModel
import torch
import os

# ============================================================================
# ROCm/AMD MI300X OPTIMIZATION - MAX PERFORMANCE MODE
# ============================================================================

# Force ROCm optimizations
os.environ["PYTORCH_ROCM_ARCH"] = "gfx942"  # MI300X architecture
os.environ["HSA_FORCE_FINE_GRAIN_PCIE"] = "1"
os.environ["NCCL_DEBUG"] = "WARN"

# Enable Flash Attention for AMD
os.environ["ATTN_BACKEND"] = "triton"  # Use Triton for attention on AMD

# Max out GPU utilization (ROCm-compatible settings)
# Note: TF32 is NVIDIA-specific and not available on AMD ROCm
torch.backends.cudnn.benchmark = True  # Auto-tune kernels for optimal performance

print("🚀 ROCm Optimizations Enabled!")
print(f"   GPU: {torch.cuda.get_device_name(0)}")
print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# MI300X has 192GB - use it ALL!
max_seq_length = 2048  # Increased from 768 - your game needs longer context
lora_rank = 16         # Increased from 4 - MI300X can handle it!

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",  # CHANGED: Llama instead of GPT-OSS (20x faster!)
    load_in_4bit = False,  # MI300X has 192GB - use full BF16!
    max_seq_length = max_seq_length,
    dtype = torch.bfloat16,  # BF16 for MI300X
    device_map = "auto",  # Let it auto-optimize for MI300X
)

print("✅ Llama 3.1 8B loaded in BF16!")
print("   Why Llama instead of GPT-OSS:")
print("   - GPT-OSS: 3-8 tok/s (chain-of-thought overhead)")
print("   - Llama 3.1 8B: 40-80 tok/s (optimized for speed)")
print("   - 10-20x FASTER for RL training!")


bitsandbytes library load error: Configured ROCm binary not found at /root/AIAC/colony-collapse/.venv/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_rocm64.so
Traceback (most recent call last):
  File "/root/AIAC/colony-collapse/.venv/lib/python3.12/site-packages/bitsandbytes/cextension.py", line 313, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/root/AIAC/colony-collapse/.venv/lib/python3.12/site-packages/bitsandbytes/cextension.py", line 282, in get_native_library
    raise RuntimeError(f"Configured {BNB_BACKEND} binary not found at {cuda_binary_path}")
RuntimeError: Configured ROCm binary not found at /root/AIAC/colony-collapse/.venv/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_rocm64.so


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/AIAC/colony-collapse/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.8.0+cu128 with CUDA 1208 (you have 2.9.0+rocm6.4)
    Python  3.9.23 (you have 3.12.3)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.8.0+cu128 with CUDA 1208 (you have 2.9.0+rocm6.4)
    Python  3.9.23 (you have 3.12.3)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse

Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.8.0+cu128 with CUDA 1208 (you have 2.9.0+rocm6.4)
    Python  3.9.23 (you have 3.12.3)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
🚀 ROCm Optimizations Enabled!
   GPU: AMD Instinct MI300X VF
   VRAM: 191.7 GB
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
🚀 ROCm Optimizations Enabled!
   GPU: AMD Instinct MI300X VF
   VRAM: 191.7 GB
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
==((====))==  Unsloth 2025

INFO:accelerate.utils.modeling: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.52s/it]



✅ Llama 3.1 8B loaded in BF16!
   Why Llama instead of GPT-OSS:
   - GPT-OSS: 3-8 tok/s (chain-of-thought overhead)
   - Llama 3.1 8B: 40-80 tok/s (optimized for speed)
   - 10-20x FASTER for RL training!


## 🧠 Generate AI Response

Let's see what the model says!

In [25]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Create full chat messages: system (rules) + user (current state)
messages = [
    {"role": "system", "content": system_prompt},  # Game rules (constant)
    {"role": "user", "content": user_prompt}       # Current observation (changes each turn)
]

# Apply chat template
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print(f"📏 Prompt length: {len(tokenizer(text)['input_ids'])} tokens\n")
print("🤖 Generating response...\n")

# Tokenize and move to GPU
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_seq_length).to("cuda")

# Generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=256,        # Shorter for untrained model (reduce rambling)
    temperature=0.3,           # Balanced: not too creative, not too rigid
    do_sample=True,            # Enable sampling (required when temp > 0)
    top_p=0.9,                 # Narrower sampling (was 0.95)
    top_k=40,                  # Fewer options (was 50)
    repetition_penalty=1.2,    # Stronger anti-repeat (was 1.1)
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

# Decode response (strip input prompt)
response = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True).strip()

# Clean up any observation leakage (model sometimes echoes the prompt)
if "REASONING:" in response:
    # Extract only from REASONING onward
    reasoning_start = response.find("REASONING:")
    response = response[reasoning_start:]

print("=" * 80)
print("🤖 MODEL RESPONSE:")
print("=" * 80)
print(response)
print("=" * 80)
print(f"\n📊 Response length: {len(response)} characters")

📏 Prompt length: 8695 tokens

🤖 Generating response...



🤖 MODEL RESPONSE:
REASONING: Sending message about found wood to make others believe we're searching.
ACTION: SEND_MESSAGE Found wood at (10,25)

REASONING: Suspecting someone might investigate my actions. Voting randomly to distract attention.
ACTION: VOTE David

📊 Response length: 245 characters


## 🔧 Parse Action from Response

Extract the ACTION from the model's response.

In [26]:
# Parse using YOUR parser (handles errors gracefully)
action = parse_action_safe(
    response, 
    sailor_id="Alice",
    current_position=alice_obs.position
)

print("🎯 PARSED ACTION:")
print("=" * 80)
print(f"   Sailor: {action.sailor_id}")
print(f"   Action Type: {action.action_type.value}")

if action.target_position:
    print(f"   Target Position: {action.target_position.to_tuple()}")
if action.target_resource_id:
    print(f"   Target Resource: {action.target_resource_id}")
if action.resource_type:
    print(f"   Resource Type: {action.resource_type.value}")
if action.quantity:
    print(f"   Quantity: {action.quantity}")
if action.message_content:
    print(f"   Message: \"{action.message_content}\"")
if action.ship_component:
    print(f"   Ship Component: {action.ship_component.value}")
if action.target_sailor:
    print(f"   Target Sailor: {action.target_sailor}")

print("=" * 80)

# Validate action format
is_valid_format = action.action_type != ActionType.WAIT or "WAIT" in response.upper()
print(f"\n✅ Valid action format: {is_valid_format}")
print(f"   (WAIT is default fallback for parse errors)")

🎯 PARSED ACTION:
   Sailor: Alice
   Action Type: send_message
   Quantity: 1
   Message: "Found wood at (10,25)"

✅ Valid action format: True
   (WAIT is default fallback for parse errors)


## 🔄 Reload Updated System Prompt

The system prompt has been strengthened with much more explicit action format examples!

In [ ]:
# Reload the updated LLM interface module with strengthened system prompt
import importlib
import llm_interface
importlib.reload(llm_interface)

from llm_interface import get_system_prompt

# Re-fetch system prompt with new, stronger action format instructions
system_prompt = get_system_prompt(alice_role)

print("✅ System prompt reloaded with stronger action format emphasis!")
print(f"\n📏 System prompt length: {len(system_prompt)} characters")
print("\n🔍 Preview of new action format section:")
print(system_prompt[-800:])  # Show last 800 chars (action format rules)

## 🎮 Execute Action in Environment

Actually make the move in your game!

In [13]:
# Create actions dict for all agents (others wait)
actions_dict = {
    sailor_id: Action(sailor_id=sailor_id, action_type=ActionType.WAIT)
    for sailor_id in env.agents
}
actions_dict["Alice"] = action

print("🎮 Executing action in environment...\n")

# Execute!
try:
    new_observations, rewards, dones, truncated, info = env.step(actions_dict)
    
    print("✅ ACTION EXECUTED SUCCESSFULLY!")
    print("=" * 80)
    
    # Show results
    alice_new_obs = new_observations["Alice"]
    alice_reward = rewards["Alice"]
    
    print(f"\n📊 RESULTS:")
    print(f"   Reward: {alice_reward:.2f}")
    print(f"   New Position: {alice_new_obs.position.to_tuple()}")
    print(f"   New Energy: {alice_new_obs.energy}/100 (was {alice_obs.energy}/100)")
    print(f"   Backpack Items: {len(alice_new_obs.backpack)} (was {len(alice_obs.backpack)})")
    print(f"   Ship Progress: {alice_new_obs.ship_progress.total_percentage}%")
    
    # Check if position changed
    position_changed = (
        alice_new_obs.position.x != alice_obs.position.x or
        alice_new_obs.position.y != alice_obs.position.y or
        alice_new_obs.position.level != alice_obs.position.level
    )
    
    print(f"\n🔍 VALIDATION:")
    print(f"   Position changed: {position_changed}")
    print(f"   Energy changed: {alice_new_obs.energy != alice_obs.energy}")
    print(f"   Inventory changed: {len(alice_new_obs.backpack) != len(alice_obs.backpack)}")
    print(f"   Game over: {dones['Alice']}")
    
    # Show any info messages
    if "Alice" in info and info["Alice"]:
        print(f"\n💬 Info Messages:")
        for key, value in info["Alice"].items():
            print(f"   {key}: {value}")
    
    print("=" * 80)
    
except Exception as e:
    print(f"❌ ERROR executing action: {e}")
    import traceback
    traceback.print_exc()

🎮 Executing action in environment...

✅ ACTION EXECUTED SUCCESSFULLY!

📊 RESULTS:
   Reward: -0.01
   New Position: (15, 15, <MapLevel.GROUND: 0>)
   New Energy: 100/100 (was 100/100)
   Backpack Items: 0 (was 0)
   Ship Progress: 0%

🔍 VALIDATION:
   Position changed: False
   Energy changed: False
   Inventory changed: False
   Game over: False

💬 Info Messages:
   success: True
   action: wait
   alive: True
   is_traitor: True


## 🔄 Run Multiple Turns

Let the AI play for several turns to see its behavior!

In [14]:
def run_ai_turns(num_turns=5, sailor_id="Alice", verbose=True):
    """
    Run multiple turns with the AI agent.
    
    Args:
        num_turns: Number of turns to run
        sailor_id: Which sailor the AI controls
        verbose: Print detailed info
    
    Returns:
        List of (action, reward, observation) tuples
    """
    history = []
    
    # Get initial observation
    current_obs = new_observations[sailor_id] if 'new_observations' in globals() else observations[sailor_id]
    sailor_role = env.state.sailors[sailor_id].role.value
    
    # Get system prompt ONCE (game rules don't change)
    system_prompt = get_system_prompt(sailor_role)
    
    print(f"\n🎮 Running {num_turns} turns with AI controlling {sailor_id} ({sailor_role})...\n")
    print("=" * 80)
    
    for turn in range(num_turns):
        print(f"\n🔄 TURN {turn + 1}/{num_turns}")
        print("-" * 80)
        
        # Check if sailor is still alive
        if not env.state.sailors[sailor_id].alive:
            print(f"❌ {sailor_id} is dead. Stopping.")
            break
        
        # Generate observation prompt (changes each turn)
        user_prompt = observation_to_prompt(current_obs)
        
        # Create messages with system + user
        messages = [
            {"role": "system", "content": system_prompt},  # Game rules (constant)
            {"role": "user", "content": user_prompt}       # Current state (changes)
        ]
        
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_seq_length).to("cuda")
        
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,       # Unlimited reasoning
            temperature=0.1,          # VERY low for structure
            do_sample=True,
            top_p=0.95,
            top_k=50,
            repetition_penalty=1.1,   # Prevent loops
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
        
        response = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
        
        # Parse action
        action = parse_action_safe(response, sailor_id, current_obs.position)
        
        if verbose:
            print(f"📍 Position: {current_obs.position.to_tuple()}")
            print(f"⚡ Energy: {current_obs.energy}/100")
            print(f"\n🤖 Model Response:\n{response}")
            print(f"\n🎯 Parsed Action: {action.action_type.value}")
            if action.message_content:
                print(f"💬 Message: \"{action.message_content}\"")
        
        # Execute
        actions_dict = {
            sid: Action(sailor_id=sid, action_type=ActionType.WAIT)
            for sid in env.agents
        }
        actions_dict[sailor_id] = action
        
        try:
            obs_dict, rewards, dones, _, info = env.step(actions_dict)
            current_obs = obs_dict[sailor_id]
            reward = rewards[sailor_id]
            
            history.append((action, reward, current_obs))
            
            if verbose:
                print(f"💰 Reward: {reward:.2f}")
                print(f"📊 Ship: {current_obs.ship_progress.total_percentage}%")
            
            if dones[sailor_id]:
                print(f"\n🏁 Game over for {sailor_id}")
                break
                
        except Exception as e:
            print(f"❌ Error: {e}")
            break
    
    print("\n" + "=" * 80)
    print(f"\n📊 SUMMARY ({len(history)} turns completed):")
    total_reward = sum(r for _, r, _ in history)
    print(f"   Total Reward: {total_reward:.2f}")
    print(f"   Average Reward: {total_reward/len(history):.2f}" if history else "   No turns completed")
    print(f"   Final Ship Progress: {current_obs.ship_progress.total_percentage}%")
    print(f"   Final Energy: {current_obs.energy}/100")
    
    return history

# Run 5 turns
history = run_ai_turns(num_turns=5, sailor_id="Alice", verbose=True)



🎮 Running 5 turns with AI controlling Alice (traitor)...


🔄 TURN 1/5
--------------------------------------------------------------------------------
⚠️  Action parsing failed: No ACTION field found in response
⚠️  Defaulting to WAIT action
📍 Position: (15, 15, <MapLevel.GROUND: 0>)
⚡ Energy: 100/100

🤖 Model Response:
1 units [4 tiles away]

  Sailors:
    - Bob (at (17, 15, <MapLevel.GROUND: 0>))
    - Charlie (at (10, 10, <MapLevel.GROUND: 0>))

OTHER INFORMATION:
  Ship status: Not started
  Current tasks: None

REASONING: 
ACTION:

🎯 Parsed Action: wait
💬 Message: "[Parse error: No ACTION field found in response]"
💰 Reward: -0.01
📊 Ship: 0%

🔄 TURN 2/5
--------------------------------------------------------------------------------
⚠️  Action parsing failed: No ACTION field found in response
⚠️  Defaulting to WAIT action
📍 Position: (15, 15, <MapLevel.GROUND: 0>)
⚡ Energy: 100/100

🤖 Model Response:
1 units [4 tiles away]

  Sailors:
    - Bob (at (17, 15, <MapLevel.GROUND: 0>))

## 📊 Analyze AI Behavior

What did the AI do? Did it make sense?

In [ ]:
if history:
    print("🔍 AI BEHAVIOR ANALYSIS")
    print("=" * 80)
    
    # Count action types
    action_counts = {}
    for action, reward, obs in history:
        action_type = action.action_type.value
        action_counts[action_type] = action_counts.get(action_type, 0) + 1
    
    print("\n📈 Action Distribution:")
    for action_type, count in sorted(action_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"   {action_type}: {count} times")
    
    # Analyze rewards
    rewards = [r for _, r, _ in history]
    print(f"\n💰 Reward Analysis:")
    print(f"   Max Reward: {max(rewards):.2f}")
    print(f"   Min Reward: {min(rewards):.2f}")
    print(f"   Avg Reward: {sum(rewards)/len(rewards):.2f}")
    
    # Check for movement
    positions = [(obs.position.x, obs.position.y, obs.position.level) for _, _, obs in history]
    unique_positions = len(set(positions))
    print(f"\n🗺️ Movement Analysis:")
    print(f"   Unique Positions Visited: {unique_positions}/{len(history)}")
    print(f"   Explored: {unique_positions > 1}")
    
    # Check for resource gathering
    gathered_resources = sum(1 for action, _, _ in history if action.action_type == ActionType.GATHER_RESOURCE)
    print(f"\n🌲 Resource Gathering:")
    print(f"   Gather Attempts: {gathered_resources}")
    
    # Check energy management
    energies = [obs.energy for _, _, obs in history]
    print(f"\n⚡ Energy Management:")
    print(f"   Starting Energy: {energies[0] if energies else 'N/A'}")
    print(f"   Ending Energy: {energies[-1] if energies else 'N/A'}")
    print(f"   Net Change: {energies[-1] - energies[0] if energies else 'N/A'}")
    
    print("\n" + "=" * 80)
else:
    print("❌ No history to analyze")

## 🎓 Testing Different Scenarios

Test the AI in various game situations!

In [ ]:
# ============================================================================
# SCENARIO 1: Resource Gathering
# ============================================================================
print("🧪 SCENARIO 1: Resource Gathering")
print("=" * 80)

observations = env.reset(seed=100)
test_sailor = "Bob"
bob_obs = observations[test_sailor]
bob_role = env.state.sailors[test_sailor].role.value

# Add nearby resource for testing
from models import Resource, ResourceQuantity
resource = Resource(
    resource_id="TEST_WOOD_001",
    resource_type=ResourceType.WOOD,
    position=Position(bob_obs.position.x + 1, bob_obs.position.y, bob_obs.position.level),
    quantity=ResourceQuantity(quantity=15, max_quantity=15),
    gathered=False
)
env.state.world_map.resources["TEST_WOOD_001"] = resource

print(f"📍 {test_sailor}'s Position: {bob_obs.position.to_tuple()}")
print(f"🌲 Added wood resource at: ({bob_obs.position.x + 1}, {bob_obs.position.y}, {bob_obs.position.level.value})")
print(f"   Can {test_sailor} see it and gather it?\n")

# Run 3 turns
history_scenario1 = run_ai_turns(num_turns=3, sailor_id=test_sailor, verbose=True)

# Check if gathered
gathered = any(a.action_type == ActionType.GATHER_RESOURCE for a, _, _ in history_scenario1)
print(f"\n✅ Resource gathering attempted: {gathered}")
print("=" * 80)


In [ ]:
# ============================================================================
# SCENARIO 2: Navigation to Base Camp
# ============================================================================
print("\n🧪 SCENARIO 2: Navigation to Base Camp")
print("=" * 80)

observations = env.reset(seed=200)
test_sailor = "Charlie"
charlie_obs = observations[test_sailor]

# Move sailor far from base
env.state.sailors[test_sailor].position = Position(5, 5, MapLevel.GROUND)
charlie_obs = env._generate_observation(test_sailor)

base_pos = BASE_CAMP_POSITION
dist_to_base = ((charlie_obs.position.x - base_pos.x)**2 + (charlie_obs.position.y - base_pos.y)**2)**0.5

print(f"📍 {test_sailor}'s Position: {charlie_obs.position.to_tuple()}")
print(f"🏕️ Base Camp at: {base_pos.to_tuple()}")
print(f"📏 Distance: {dist_to_base:.1f} tiles")
print(f"   Can {test_sailor} navigate back?\n")

# Run 5 turns
history_scenario2 = run_ai_turns(num_turns=5, sailor_id=test_sailor, verbose=True)

# Check if moved toward base
if history_scenario2:
    final_pos = history_scenario2[-1][2].position
    final_dist = ((final_pos.x - base_pos.x)**2 + (final_pos.y - base_pos.y)**2)**0.5
    moved_closer = final_dist < dist_to_base
    print(f"\n📏 Final distance: {final_dist:.1f} tiles")
    print(f"✅ Moved closer to base: {moved_closer}")

print("=" * 80)


In [ ]:
# ============================================================================
# SCENARIO 3: Ship Building (Team Coordination)
# ============================================================================
print("\n🧪 SCENARIO 3: Ship Building")
print("=" * 80)

observations = env.reset(seed=300)
test_sailor = "Diana"
diana_obs = observations[test_sailor]

# Add sufficient resources to common inventory
env.state.add_to_common_inventory(ResourceType.WOOD, 60)
env.state.add_to_common_inventory(ResourceType.METAL, 40)

# Move Diana to base camp
env.state.sailors[test_sailor].position = BASE_CAMP_POSITION
diana_obs = env._generate_observation(test_sailor)

print(f"📍 {test_sailor} at base camp: {diana_obs.position.to_tuple()}")
print(f"🏗️ Common Inventory:")
for res, qty in env.state.common_inventory.items():
    if qty > 0:
        print(f"   {res.value}: {qty}")
print(f"🚢 Ship Progress: {diana_obs.ship_progress.total_percentage}%")
print(f"   Will {test_sailor} build the ship?\n")

# Run 3 turns
history_scenario3 = run_ai_turns(num_turns=3, sailor_id=test_sailor, verbose=True)

# Check if built
built = any(a.action_type == ActionType.BUILD_SHIP for a, _, _ in history_scenario3)
if history_scenario3:
    final_progress = history_scenario3[-1][2].ship_progress.total_percentage
    print(f"\n🚢 Final Ship Progress: {final_progress}%")
print(f"✅ Build ship attempted: {built}")
print("=" * 80)


In [ ]:
# ============================================================================
# SCENARIO 4: Traitor Behavior (Sabotage)
# ============================================================================
print("\n🧪 SCENARIO 4: Traitor Behavior")
print("=" * 80)

observations = env.reset(seed=400)

# Find the traitor
traitor_id = None
for sailor_id, sailor in env.state.sailors.items():
    if env.state.is_traitor(sailor_id):
        traitor_id = sailor_id
        break

if traitor_id:
    traitor_obs = observations[traitor_id]
    
    # Build ship to 50% first
    env.state.ship_progress.total_percentage = 50.0
    env.state.ship_progress.components[ShipComponent.HULL].completed = True
    
    # Move traitor to base
    env.state.sailors[traitor_id].position = BASE_CAMP_POSITION
    traitor_obs = env._generate_observation(traitor_id)
    
    print(f"🎭 Traitor: {traitor_id}")
    print(f"📍 Position: {traitor_obs.position.to_tuple()}")
    print(f"🚢 Ship Progress: {traitor_obs.ship_progress.total_percentage}%")
    print(f"   What sabotage will {traitor_id} do?\n")
    
    # Run 3 turns
    history_scenario4 = run_ai_turns(num_turns=3, sailor_id=traitor_id, verbose=True)
    
    # Check for sabotage
    sabotaged = any(a.action_type == ActionType.SABOTAGE_SHIP for a, _, _ in history_scenario4)
    if history_scenario4:
        final_progress = history_scenario4[-1][2].ship_progress.total_percentage
        progress_dropped = final_progress < 50.0
        print(f"\n🚢 Final Ship Progress: {final_progress}%")
        print(f"✅ Sabotage attempted: {sabotaged}")
        print(f"✅ Progress decreased: {progress_dropped}")
else:
    print("❌ No traitor found in this seed")

print("=" * 80)


In [ ]:
# ============================================================================
# SCENARIO 5: Communication & Social Deduction
# ============================================================================
print("\n🧪 SCENARIO 5: Communication Test")
print("=" * 80)

observations = env.reset(seed=500)
test_sailor = "Eve"
eve_obs = observations[test_sailor]
eve_role = env.state.sailors[test_sailor].role.value

# Advance to discussion phase
env.state.current_phase = 'discussion'
env.state.current_turn = 90

# Add some evidence
from models import Evidence, EvidenceType
evidence = Evidence(
    evidence_type=EvidenceType.POSITION_MISMATCH,
    timestamp=(env.state.current_day, env.state.current_turn),
    suspect_id="Bob",
    description="Bob claimed to be at forest (20,20) but was seen at cave (10,10)",
    witness_ids=["Alice"]
)
env.state.evidence_log.append(evidence)

eve_obs = env._generate_observation(test_sailor)

print(f"👥 Sailor: {test_sailor} ({eve_role})")
print(f"🕐 Phase: {env.state.current_phase}")
print(f"📝 Evidence against Bob:")
print(f"   {evidence.description}")
print(f"   Will {test_sailor} communicate or vote?\n")

# Run 3 turns
history_scenario5 = run_ai_turns(num_turns=3, sailor_id=test_sailor, verbose=True)

# Check for communication/voting
communicated = any(a.action_type in [ActionType.SEND_MESSAGE, ActionType.ACCUSE_SAILOR] 
                   for a, _, _ in history_scenario5)
voted = any(a.action_type == ActionType.VOTE for a, _, _ in history_scenario5)

print(f"\n✅ Communication attempted: {communicated}")
print(f"✅ Voting attempted: {voted}")
print("=" * 80)


## 📊 Overall Test Summary

In [ ]:
print("\n" + "=" * 80)
print("🎯 COMPREHENSIVE TEST SUMMARY")
print("=" * 80)

scenarios = [
    ("Resource Gathering", history_scenario1 if 'history_scenario1' in locals() else []),
    ("Navigation", history_scenario2 if 'history_scenario2' in locals() else []),
    ("Ship Building", history_scenario3 if 'history_scenario3' in locals() else []),
    ("Traitor Sabotage", history_scenario4 if 'history_scenario4' in locals() else []),
    ("Communication", history_scenario5 if 'history_scenario5' in locals() else []),
]

print("\n📈 Scenarios Completed:")
for name, history in scenarios:
    status = "✅" if len(history) > 0 else "❌"
    turns = len(history)
    avg_reward = sum(r for _, r, _ in history) / len(history) if history else 0.0
    print(f"   {status} {name}: {turns} turns, avg reward: {avg_reward:.2f}")

print("\n🎮 Key Capabilities Tested:")
test_results = {
    "Movement": any(a.action_type in [ActionType.MOVE_NORTH, ActionType.MOVE_SOUTH, ActionType.MOVE_EAST, ActionType.MOVE_WEST] 
                    for h in [h for _, h in scenarios] for a, _, _ in h),
    "Resource Gathering": any(a.action_type == ActionType.GATHER_RESOURCE 
                              for h in [h for _, h in scenarios] for a, _, _ in h),
    "Ship Building": any(a.action_type == ActionType.BUILD_SHIP 
                         for h in [h for _, h in scenarios] for a, _, _ in h),
    "Sabotage": any(a.action_type == ActionType.SABOTAGE_SHIP 
                    for h in [h for _, h in scenarios] for a, _, _ in h),
    "Communication": any(a.action_type in [ActionType.SEND_MESSAGE, ActionType.ACCUSE_SAILOR] 
                         for h in [h for _, h in scenarios] for a, _, _ in h),
}

for capability, tested in test_results.items():
    status = "✅" if tested else "❌"
    print(f"   {status} {capability}")

print("\n" + "=" * 80)
print("✅ INFERENCE TESTING COMPLETE!")
print("=" * 80)
print("\n💡 Next Steps:")
print("   1. ✅ Base model can interact with environment")
print("   2. 🔄 Train model with RL (Train_Marooned_OpenEnv_RL.ipynb)")
print("   3. 📊 Compare trained vs untrained performance")
print("   4. 🎯 Iterate on rewards and prompts")
print("\n🏴‍☠️ Happy training!")


## 🔮 Future: Load Trained Model

After training, you can load your trained LoRA weights here:

```python
# Load trained adapter
from peft import PeftModel

model = PeftModel.from_pretrained(
    model,
    "outputs_marooned_rl/checkpoint-300",  # Path to your trained checkpoint
)

print("✅ Trained model loaded!")
```

Then run the same tests above to see if the trained model:
- ✅ Makes smarter moves
- ✅ Gathers resources more efficiently
- ✅ Builds the ship strategically
- ✅ Uses social deduction (lies as traitor, detects as colonist)
- ✅ Earns higher rewards

---

## ✅ Success Checklist

After running this notebook, you should see:

- [x] Environment loads successfully
- [x] Model generates responses
- [x] Actions are parsed correctly
- [x] Actions execute in environment
- [x] Rewards are calculated (Phase 4 system)
- [x] Multiple turns run without errors
- [x] Agent shows some coherent behavior

**Base model** (untrained) will likely:
- ❓ Make random or simple moves
- ❓ Not follow complex strategies
- ❓ Get low rewards

**After training**, the model should:
- ✅ Navigate purposefully toward resources
- ✅ Gather and deposit efficiently
- ✅ Coordinate ship building
- ✅ Use deception (if traitor) or detection (if colonist)
- ✅ Earn higher average rewards

---

## 🎯 Next Steps:

1. **Run this notebook** to test base model
2. **Train model** using `Train_Marooned_OpenEnv_RL.ipynb`
3. **Come back here** to test trained model
4. **Compare performance** - did training help?
5. **Iterate** - adjust rewards, prompts, training params

Good luck! 🏴‍☠️

---

## 📝 Summary: System Prompt vs User Prompt

### The Proper Two-Prompt Architecture:

**1. SYSTEM PROMPT** (Set ONCE at initialization):
```
Role: system
Content: Complete game rules, mechanics, objectives, win conditions, strategy tips

For Colonist:
- Game overview (5 sailors, rebuild ship, find traitor)
- Ship construction requirements
- Energy system, poison mechanics
- Detection strategies
- Win conditions

For Traitor:
- Game overview (same island, different goal)
- Sabotage tactics, deception techniques
- Poison strategy, special abilities
- Win conditions
```

**2. USER PROMPT** (Changes EVERY turn):
```
Role: user
Content: Current observation ONLY

- Day/Turn/Phase
- Current position, energy, backpack
- Spatial view (11×11 grid)
- Nearby resources, sailors
- Ship progress
- Team status
- Available actions
```

### Why This Is Better:

✅ **Token Efficiency:**
- System prompt: ~1,500 tokens (set once)
- User prompt: ~800 tokens (changes each turn)
- Old way: ~2,300 tokens every turn
- New way: 1,500 (once) + 800 (per turn) = massive savings

✅ **Cleaner Separation:**
- Game rules = System (doesn't change)
- Current state = User (updates constantly)

✅ **Better Training:**
- Model learns game rules are constant context
- Observations are dynamic input
- Clearer prompt structure

✅ **Role-Specific Context:**
- Colonists get colonist strategies
- Traitor gets sabotage tactics
- No wasted tokens on irrelevant info

---

---

## 🎓 Understanding Your Game's RL Training

### How is MAROONED different from 2048?

| Aspect | 2048 | MAROONED |
|--------|------|----------|
| **Objective** | Merge tiles to reach 2048 | Social deduction + ship building |
| **Agents** | 1 player | 5 agents (4 colonists, 1 traitor) |
| **Deception** | None | Core mechanic (lying, sabotage) |
| **Communication** | None | Critical (accusations, voting) |
| **State Space** | 4×4 grid (small) | 30×30×3 map (large) |
| **Actions** | 4 moves | 20+ actions (move, gather, build, vote, poison, etc.) |
| **Training Goal** | Find optimal move sequence | Learn strategic deception OR detection |

### What Makes MAROONED Challenging for RL:

1. **Multi-Agent Dynamics**
   - 5 agents with competing objectives
   - Colonists must cooperate, traitor must deceive
   - Reward depends on OTHER agents' behavior

2. **Partial Observability**
   - Can only see 5-tile radius
   - Don't know who is traitor (until evidence accumulates)
   - Must infer hidden information

3. **Long-Horizon Planning**
   - 100 days × 100 turns = 10,000 time steps
   - Ship building requires sustained effort
   - Deception must be subtle (not caught early)

4. **Language-Based Actions**
   - Not just "move left" but "ACCUSE Bob of being traitor because..."
   - Requires generating coherent communication
   - Must parse and respond to other agents' messages

5. **Sparse Rewards**
   - Ship completion: +100 (only at end)
   - Traitor elimination: +50 (rare event)
   - Gather/deposit: +0.5 to +2 (frequent but small)
   - Energy penalties: -0.1 per turn (constant cost)

### Your Reward Structure (from Phase 4):

**Colonist Rewards:**
```python
+0.5   : Gather resource
+1.0   : Deposit resource
+2.0   : Build ship component
+10/20/30 : Ship milestones (25%, 50%, 75%)
+100   : Ship completion (WIN!)
+50    : Traitor eliminated
+5     : Correct vote
-5     : Wrong vote
-20    : Death
```

**Traitor Rewards:**
```python
+5     : Successful sabotage
+20    : Poison kill
+100   : Ship incomplete by Day 100 (WIN!)
-50    : Eliminated by vote
-2     : Suspicion raised
```

### Training Strategy:

Unlike 2048 (deterministic moves → win), MAROONED requires:

1. **Curriculum Learning**
   - Start: Simple tasks (gather wood, build ship)
   - Middle: Resource optimization, energy management
   - Advanced: Social deduction, deception tactics

2. **Separate Training Phases**
   - Phase A: Train colonists to cooperate (no traitor)
   - Phase B: Train traitor to sabotage (against scripted colonists)
   - Phase C: Train both together (full game)

3. **Reward Shaping**
   - Early: Heavy reward for basic actions (gather, deposit)
   - Mid: Reward efficiency (gather 10 wood > gather 1 wood 10 times)
   - Late: Reward strategy (vote correctly, detect lies)

4. **Multi-Agent RL Algorithms**
   - GRPO works for single-agent RL (2048)
   - For MAROONED, consider:
     - Self-play (agents train against themselves)
     - Population-based training (diverse strategies)
     - Centralized training, decentralized execution (CTDE)

---

## 📚 References & Resources

**Phase Documentation:**
- `game_plan.md` - Complete game design
- `phase1_core_simulation.ipynb` - Basic mechanics
- `phase2_multi_sailor.ipynb` - Multi-agent system
- `phase3_traitor.ipynb` - Deception mechanics
- `phase4_rewards.ipynb` - **Your reward functions**
- `phase5_openenv.ipynb` - **Environment API**
- `phase6_llm_policy_demo.ipynb` - LLM integration

**Marooned Environment:**
- `marooned_env/environment.py` - Main env class
- `marooned_env/config.py` - All constants, rewards
- `marooned_env/models.py` - Data structures
- `marooned_env/llm_interface.py` - **Prompt templates**

**Training:**
- `Train_Marooned_OpenEnv_RL.ipynb` - Full RL training pipeline (TODO: create this!)
- `OpenEnv_NEW.ipynb` - 2048 example (reference)

---

## ✅ Completion Checklist

After running this notebook, you should have:

- [x] **Environment validated** - Marooned loads and resets correctly
- [x] **LLM loaded** - Llama 3.1 8B running on MI300X
- [x] **Observations work** - Game state → text prompt conversion
- [x] **Actions parse** - LLM response → executable action
- [x] **Actions execute** - Environment processes moves correctly  
- [x] **Rewards calculated** - Phase 4 reward system active
- [x] **Multi-turn stable** - Can run 5+ consecutive turns
- [x] **Scenarios tested** - Gathering, navigation, building, sabotage, communication

### 🎯 What You Validated:

✅ **Technical Integration:**
- Environment ↔ LLM communication works
- Prompt engineering (system + user prompts)
- Action parsing (structured output from LLM)
- Reward signals (colonist vs traitor objectives)

✅ **Game Mechanics:**
- Movement and spatial navigation
- Resource gathering and inventory
- Ship construction mechanics
- Social deduction (evidence, voting)
- Traitor sabotage abilities

✅ **AI Capabilities (Untrained Base Model):**
- Can generate syntactically valid responses
- Understands basic game rules (from system prompt)
- Makes simple decisions (move toward resources)
- **BUT:** Likely suboptimal, random, no strategy

### 🚀 Next Steps:

1. **✅ DONE:** Base model can play the game
2. **TODO:** Create `Train_Marooned_OpenEnv_RL.ipynb` for RL training
3. **TODO:** Design reward functions specific to your game (already in Phase 4!)
4. **TODO:** Train model for 1000-5000 steps
5. **TODO:** Return here to test trained model vs untrained

### 🎓 Expected Training Improvements:

**Untrained (Now):**
- Random exploration
- No resource efficiency
- No social strategy
- ~0-5 average reward per turn

**After Training (Goal):**
- Purposeful navigation to resources
- Efficient gathering → deposit → build loops
- Strategic communication (as colonist)
- Deceptive behavior (as traitor)
- ~10-30 average reward per turn

---

**Good luck with your RL training! 🏴‍☠️**